In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

c:\Users\zbc44\anaconda3\envs\data100\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [2]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
181561,CA,F,2008,Lisa,73
167122,CA,F,2004,Ashna,7
104282,CA,F,1987,Clarice,11
133814,CA,F,1995,Jason,7
225706,CA,F,2019,Opal,39


Goal 1: Find the most popular baby name in California in 2018

In [3]:
babynames[(babynames["Year"] == 2018) & (babynames['Sex'] == 'F')].sort_values(by = "Count", ascending = False).head(5)

,State,Sex,Year,Name,Count
221179,CA,F,2018,Emma,2746
221180,CA,F,2018,Mia,2502
221181,CA,F,2018,Olivia,2467
221182,CA,F,2018,Isabella,2202
221183,CA,F,2018,Sophia,2154


Goal 2: Baby names that start with j. 

In [4]:
starts_with_j = babynames["Name"].str.startswith('J')
starts_with_j.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: Name, dtype: bool

In [5]:
pd.DataFrame(babynames[starts_with_j]["Name"].unique()).sample(10)

,0
1346,Joseandres
1541,Joven
1234,Jeanluc
607,Jezebel
662,Journee
352,Jessenia
910,Jayliana
962,Jim
1212,Jushua
449,Jayde


Goal 2: Sort names by their length.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [6]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].map(len)

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length",axis=1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count
376803,CA,M,2013,An,7
338135,CA,M,1999,Cj,6
264612,CA,M,1957,Ed,53
308634,CA,M,1987,Lo,6
356854,CA,M,2006,Aj,5


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [7]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].str.len()

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length", 1)
babynames_by_length.head(5)

C:\Users\zbc44\AppData\Local\Temp\ipykernel_9008\3542196870.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  babynames_by_length = babynames_by_length.drop("name_length", 1)


,State,Sex,Year,Name,Count
376803,CA,M,2013,An,7
338135,CA,M,1999,Cj,6
264612,CA,M,1957,Ed,53
308634,CA,M,1987,Lo,6
356854,CA,M,2006,Aj,5


Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [13]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count,name_length
376803,CA,M,2013,An,7,2
338135,CA,M,1999,Cj,6,2
264612,CA,M,1957,Ed,53,2
308634,CA,M,1987,Lo,6,2
356854,CA,M,2006,Aj,5,2


How does this work exactly? Let's break it into pieces.

In [8]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

376803    2
338135    2
264612    2
308634    2
356854    2
Name: Name, dtype: int64

In [9]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

Int64Index([376803, 338135, 264612, 308634, 356854, 308613, 295333, 321001,
            249345, 338172,
            ...
            308802, 313143, 340695, 318170, 330421, 333732, 333556, 318049,
            310235, 102498],
           dtype='int64', length=400762)

In [14]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture 5.
babynames.loc[index_sorted_by_length].head(5)

,State,Sex,Year,Name,Count,name_length
376803,CA,M,2013,An,7,2
338135,CA,M,1999,Cj,6,2
264612,CA,M,1957,Ed,53,2
308634,CA,M,1987,Lo,6,2
356854,CA,M,2006,Aj,5,2


Goal 4: Name whose popularity has changed the most. Also tough.

First we need to define change in popularity. For simplicity, for this problem, let's define it as the difference between a name's maximum occurrence and minimum occurrence. For example, let's consider the name Jennifer.

In [15]:
jennifer_counts = babynames.query("Name == 'Jennifer'")["Count"]
jennifer_counts.head(5)

13610     5
16325     5
16993     6
17533    13
18210    24
Name: Count, dtype: int64

In [16]:
max(jennifer_counts) - min(jennifer_counts)

6060

In [17]:
def domain_size(series):
    return max(series) - min(series)

In [18]:
domain_size(jennifer_counts)

6060

In [19]:
name_domain_sizes = babynames.groupby("Name").agg(domain_size)
name_domain_sizes.head(5)

C:\Users\zbc44\AppData\Local\Temp\ipykernel_9008\165560736.py:1: FutureWarning: ['State', 'Sex'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  name_domain_sizes = babynames.groupby("Name").agg(domain_size)


,Year,Count,name_length
Name,,,
Aadan,6,2,0
Aadarsh,0,0,0
Aaden,13,148,0
Aadhav,5,2,0
Aadhira,4,4,0


In [20]:
name_domain_sizes.sort_values("Count", ascending=False)

,Year,Count,name_length
Name,,,
Michael,111,8257,0
David,111,8111,0
John,111,7319,0
Robert,111,7093,0
Jessica,84,6947,0
...,...,...,...
Raniya,6,0,0
Hallel,0,0,0
Ranesha,0,0,0


These goals are hard with our tools so far. Will discuss next ime.